<a href="https://colab.research.google.com/github/fkatelyn/congressional2023/blob/main/notebooks/palm_tree_gradio_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
MODEL_VERSION = '7'
DEMO = True

In [ ]:

!pip install ultralytics==8.0.134
!pip install gradio

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()





Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/166.8 GB disk)


In [ ]:
import os
HOME = os.getcwd()
HOME = '/content'
print(HOME)

from google.colab import drive
GOOGLE_DRIVE = f'{HOME}/drive'
drive.mount(GOOGLE_DRIVE)

/content
Mounted at /content/drive


In [ ]:
class Experiment:
  '''
  Google drive path:

  PROJECT_DRIVE = MyDrive/ai/projects/palmtree
  EXPERIMENT_DRIVE = MyDrive/ai/projects/palmtree/experiments
  UPLOAD_DRIVE = MyDrive/ai/projects/palmtree/uploads
  DATASET_DRIVE = MyDrive/ai/projects/palmtree/datasets
  DEMO_IMAGES_DRIVE = MyDrive/ai/projects/palmtree/demo/images

  '''
  EPOCHS = 200
  IMGSZ = 640
  CONFIDENT = 0.15
  MODEL_NAME = "yolov8s"
  EXPERIMENT_HOME = f'{HOME}/experiments/{MODEL_VERSION}'
  DATASET_HOME = f'{HOME}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_HOME = f'{HOME}/weights/{MODEL_VERSION}'
  DEMO_HOME = f'{HOME}/demo'
  IMAGES_DEMO_HOME = f'{DEMO_HOME}/images'
  VIDEOS_DEMO_HOME = f'{DEMO_HOME}/videos'


  PROJECT_DRIVE = f'{GOOGLE_DRIVE}/MyDrive/ai/projects/palmtree'
  EXPERIMENT_DRIVE = f'{PROJECT_DRIVE}/experiments/{MODEL_VERSION}'
  DATASET_DRIVE = f'{PROJECT_DRIVE}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_DRIVE = f'{PROJECT_DRIVE}/weights/{MODEL_VERSION}'
  DEMO_DRIVE = f'{PROJECT_DRIVE}/demo'
  IMAGES_DEMO_DRIVE = f'{DEMO_DRIVE}/images'
  VIDEOS_DEMO_DRIVE = f'{DEMO_DRIVE}/images'

  PROJECT_SHARED = f'{GOOGLE_DRIVE}/MyDrive/ai/shared/palmtree'
  DEMO_SHARED = f'{PROJECT_SHARED}/demo'
  IMAGES_DEMO_SHARED = f'{DEMO_SHARED}/images'
  VIDEOS_DEMO_SHARED = f'{DEMO_SHARED}/videos'
  WEIGHTS_SHARED = f'{PROJECT_SHARED}/weights/{MODEL_VERSION}'

  DEMO_SOURCE = DEMO_SHARED if DEMO else DEMO_DRIVE
  WEIGHTS_SOURCE = WEIGHTS_SHARED if DEMO else WEIGHTS_DRIVE


print(Experiment.EXPERIMENT_HOME)


/content/experiments/7


In [ ]:
%cd {HOME}
%mkdir -p {HOME}/weights/
%mkdir -p {HOME}/demo/

print(Experiment.WEIGHTS_SOURCE)
print(Experiment.DEMO_SOURCE)


!cp -fpR {Experiment.WEIGHTS_SOURCE} /content/weights/
!cp -fpR {Experiment.DEMO_SOURCE} /content/


/content
/content/drive/MyDrive/ai/inspiritai/palmtree/weights/7
/content/drive/MyDrive/ai/inspiritai/palmtree/demo


In [ ]:
from PIL import Image
import gradio as gr
import torch
from ultralytics import YOLO
import glob
import os
import shutil

model = YOLO(f'{Experiment.WEIGHTS_HOME}/best.pt')

def yolo_image(im, confident, size=Experiment.IMGSZ):
    g = (size / max(im.size))  # gain
    im = im.resize((int(x * g) for x in im.size), Image.LANCZOS)

    results = model.predict(im, conf=confident)
    im_array = results[0].plot()  # updates results.imgs with boxes and labels
    imo = Image.fromarray(im_array[..., ::-1])  # RGB

    return imo


def yolo_video(im, confident, size=Experiment.IMGSZ):
  print(confident)
  print(size)
  print(im)
  file_name = os.path.basename(im)
  base_name = os.path.splitext(file_name)[0]
  file_name = base_name + ".avi"

  if os.path.exists('/content/demo/runs'):
    shutil.rmtree('/content/demo/runs')

  results = model.predict(im, conf=confident, save=True, project='/content/demo', name='runs')
  final_video = f'/content/demo/runs/{file_name}'
  return final_video



gr_image = gr.Interface(
    fn=yolo_image,
    inputs=[gr.inputs.Image(type='pil', label="Original Image"),
            gr.Slider(0.0, 1.0, 0.5)],
    outputs=gr.outputs.Image(type="pil", label="Output Image"),
    title="Palm Oil Tree detection",
    description="Upload an image or click an example image to use.",
    examples=list(map(lambda x: [x], glob.glob(f'{Experiment.IMAGES_DEMO_HOME}/*.jpg'))),
    analytics_enabled=False)

gr_video = gr.Interface(
    yolo_video,
    inputs=[gr.PlayableVideo(type='file'), gr.Slider(0.0, 1.0, 0.5)],
    outputs=gr.PlayableVideo(),
    title="Palm Oil Tree detection",
    description="Upload a video or click an example video to use.",
    examples=list(map(lambda x: [x], glob.glob(f'{Experiment.VIDEOS_DEMO_HOME}/*.mp4'))),
    analytics_enabled=False)


demo = gr.TabbedInterface([gr_image, gr_video], ["Image detection", "Video detection"])
demo.launch(debug=True)

<ipython-input-6-eac29cb24b60>:41: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=[gr.inputs.Image(type='pil', label="Original Image"),
<ipython-input-6-eac29cb24b60>:41: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Image(type='pil', label="Original Image"),
<ipython-input-6-eac29cb24b60>:43: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Image(type="pil", label="Output Image"),
<ipython-input-6-eac29cb24b60>:51: GradioUnusedKwargWarning: You have unused kwarg parameters in PlayableVideo, please remove them: {'type': 'file'}
  inputs=[gr.PlayableVideo(type='file'), gr.Slider(0.0, 1.0, 0.5)],
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:956: UserWarning: api_name predict

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>


0: 640x640 28 healthys, 13 nitrogens, 16.7ms
Speed: 22.3ms preprocess, 16.7ms inference, 35.7ms postprocess per image at shape (1, 3, 640, 640)


    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs



0.5
640
/tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4


video 1/1 (1/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 13 healthys, 7 nitrogens, 67.5ms
video 1/1 (2/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 12 healthys, 8 nitrogens, 11.0ms
video 1/1 (3/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 13 healthys, 6 nitrogens, 11.0ms
video 1/1 (4/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 9 healthys, 8 nitrogens, 11.2ms
video 1/1 (5/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 10 healthys, 7 nitrogens, 11.6ms
video 1/1 (6/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 9 healthys, 6 nitrogens, 11.0ms
video 1/1 (7/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 13 healthys, 6 nitrogens, 11.0ms
video 1/1 (8/301) /tmp/gradio/35dfd6e8046f0694e7f201f083db1508836451c4/palmtree.mp4: 384x640 1 banana, 5 healthys, 6 nit

Keyboard interruption in main thread... closing server.
